In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

# Level 2: Processing CSV

**Note**  
Low level dataframes processing (like converting datetimes, filtering, dropping duplicates, sorting) is done in the previous notebook. Here we assume that the data is already cleaned and ready for further processing.